# Проект для «Викишоп» <a class = 'tocSkip'>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В проекте будет обучена модель классификации комментариев на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок. Значение метрики качества *F1* должно быть не меньше 0.75. 

## 1. Загрузка и подготовка данных.

Выведем общую информацию о ДатаСете, проверим на наличие критичных ошибок, а также оценим насколько сбалансированы классы целевого признака. Так как моделям машинного обучения предстоит работать с текстами, даже при отсутствии критичных ошибок в данных понадобится дополнительная предобработка датасета.

### 1.1. Общая информация о датасете.

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак. Импортируем библиотеки, посмотрим на наш датасет.

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import nltk
import re

# Импорт отдельных методов
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier

# Отключаем уведомления об ошибках.
import warnings
warnings.filterwarnings("ignore")

# Открываем файл, выводим общую информацию.
data = pd.read_csv('datasets/toxic_comments.csv')
print('Информация о Датасете:', data.info())
print('Количество дублей:', data.duplicated().sum())
print('---------------------------------------------------')
print('Соотношение позитивных и негативных комментариев:')
print('Положительные:', data['toxic'].value_counts()[0], 'или', 
      (data['toxic'].value_counts()[0] / data.shape[0]).round(3)*100, '%')
print('Негативные:', data['toxic'].value_counts()[1], 'или', 
      (data['toxic'].value_counts()[1] / data.shape[0]).round(3)*100, '%')
print('---------------------------------------------------')
print('Общий вид таблицы:')
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
Информация о Датасете: None
Количество дублей: 0
---------------------------------------------------
Соотношение позитивных и негативных комментариев:
Положительные: 143346 или 89.8 %
Негативные: 16225 или 10.2 %
---------------------------------------------------
Общий вид таблицы:


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


С данными в целом всё в порядке - пропусков и дублей нет, сама структура таблицы лаконична и понятна, но лишь для человека, а для работы моделей машинного обучения дополнительная предобработка всё равно потребуется. Единственный недостаток (или скорее особенность) в данных - целевой признак не сбалансирован, будем учитывать это при разделении выборок на обучающую и тестовую. Можем переходить к следующему этапу.

### 1.2. Преобразование данных

Перед тем как передать данные в виде признаков для машинного обучения, эти данные необходимо дополнительно упростить и преобразовать. 

 1. Потребуется провести Лемматизацию (с английским текстом поможет WordNetLemmatizer)
 2. От лишних символов текст очистят регулярные выражения (и соответствующая библиотека re)

In [2]:
%%time

# Создаём функцию очистки текста при помощи регулярных выражений, а также добавления лемматизации.
def clear_and_lemm(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    clear_text = " ".join(text.split())
    lemm_list = [lemmatizer.lemmatize(word) for word in clear_text.split()]
    clear_and_lemm_text = " ".join(lemm_list)  
    return clear_and_lemm_text

# Загрузка слов из библиотеки nltk
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Применение функции
data['lemm_text'] = data['text'].apply(clear_and_lemm)
print('Вид новой таблицы:')
display(data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Вид новой таблицы:


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


Wall time: 52.6 s


Текст очищен и лемматизирован, но работа с данными ещё не завершена - дополнительные манипуляции будут разными для разных моделей, поэтому производить их будем уже в рамках обучения, к которому сейчас и перейдём.

## 2. Обучение и проверка моделей.

Так как в нашем распоряжении достаточно большое количество наблюдений (комментариев), а признаки представляют собой наборы текста, работа с которыми относительно ресурсозатратна - обучение и тестирование моделей будет происходить в два этапа.

1. Сначала проверим общую работоспособность и скорость моделей без тюнинга гиперпараметров. Это позволит оценить номинальную скорость моделей, и соответственно - потенциал 
2. Подбор гиперпараметров и финальное тестирование.

### 2.1. Разделение выборок, запуск моделей без подбора гиперпараметров.

Сразу разделим датасет на обучающую и тестовую выборку, не забудем указать параметр stratify, чтобы учесть дисбаланс классов. Модели будут обучаться и подбирать параметры на обучающей выборке. Вместе с тем будем сразу проверять результат, делая предсказания на тестовой выборке.

Весь процесс будет протекать внутри заранее написанной функции поиска по сетке, которая будет подготовлена уже перед пробными запусками моделей, но на первом этапе будем передавать в функцию пустую сетку параметров. 

In [3]:
%%time
# Оставим 10 % всех комментариев для тестовой выборки.
train, test = train_test_split(data, test_size = 0.1, random_state = 321, stratify = data['toxic'])

# Определяем признаки
X_train = train['lemm_text'].values.astype('U')
y_train = train['toxic']

# "Истинными" значениями будут значения целевого признака в тестовой выборке (y_test) 
y_true = test['toxic']

# Создание списка стоп-слов, счётчика с со стоп словами 
stop_words = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

# Посчитаем TF-IDF для X_train и X_test, вызовем функцию fit_transform():
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(test['lemm_text'].values.astype('U'))

# Создаём функцию для GridSearch
def grid_search(model, param_grid):
    
    # Задаём сетку GridSearchCV
    model_Grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = 'f1')    
    
    # Обучение модели, вывод лучших параметров и качества модели
    model_Grid.fit(tf_idf_train, y_train)    
    print('Оптимальные параметры модели:')
    display(model_Grid.best_params_)
    print('Качество модели после обучения:', model_Grid.best_score_)
    predictions = model_Grid.predict(tf_idf_test)
    print('F1-score на тестовой выборке:', f1_score(y_true, predictions))
    
# Пустая сетка параметров
param_grid = {}

Wall time: 11 s


In [ ]:
%%time
print('Логистическая регрессия.')
model1 = LogisticRegression()
grid_search(model1, param_grid)

In [4]:
%%time
print('LGBMClassifier')
model2 = LGBMClassifier()
grid_search(model2, param_grid)

LGBMClassifier
Оптимальные параметры модели:


{}

Качество модели после обучения: 0.7413437907574343
F1-score на тестовой выборке: 0.7511737089201878
Wall time: 2min 16s


При попытке обучить CatBoost на тех же подготовленных признаках, что и предыдущие модели, ядро Jupyter Notebook падает, поэтому для CatBoost будут созданы отдельные типы данных Pool (для тестовой и для обучающей выборки).

In [5]:
%%time
print('CatBoostClassifier')
y_train_cb, X_train_cb = train['toxic'], train.drop(['toxic', 'lemm_text'], axis=1)
y_test_cb, X_test_cb = test['toxic'], test.drop(['toxic', 'lemm_text'], axis=1)

train_pool = Pool(data=X_train_cb, label=y_train_cb, text_features=['text'])
test_pool = Pool(data=X_test_cb, label=y_test_cb, text_features=['text'])

model3 = CatBoostClassifier(task_type='GPU', 
                            devices='0:1', 
                            verbose=False).fit(train_pool)

print('F1-score на тестовой выборке:', f1_score(y_true, model3.predict(test_pool)))

CatBoostClassifier
F1-score на тестовой выборке: 0.700525394045534
Wall time: 25 s


In [7]:
%%time
print('RandomForestClassifier')
#model4 = RandomForestClassifier()
#grid_search(model4, param_grid)
print('Был принудтельно прерван после 30 минут неуспешной работы.')

RandomForestClassifier
Был принудтельно прерван после 30 минут неуспешной работы.
Wall time: 1e+03 µs


Итоги по моделям следующие:

 - **Логистическая регрессия** традиционно стала наиболее быстрым вариантом. Ко всему прочему она показала неплохое качество, и почти сразу дотянула до требуемого качества. После подбора гиперпараметров наверняка получится добиться необходимого уровня метрики F1.
 - **LGBM** был менее расторопным на пустой сетке, но уже смог преодолеть отметку F1-score 0.75. Попробуем закрепить результат, и немного пройдёмся по сетке параметров.
 - **CatBoost** продемонстрировал наименьшее качество, но благодаря простой возможности использовать GPU обучился намного быстрее своего конкурента по градиентному бустингу. Попробуем подобрать для него параметры, тем более скорость позволяет это сделать.
 - **Случайный лес** показал неприлично долгое время работы, и после получаса незавершенной работы на пустой сетке был принудительно прерван - тут о поиске гиперпараметров даже не приходится говорить. В следующий этап модель не проходит.

### 2.2. Подбор гиперпараметров.

Попробуем улучшить полученные результаты путём подбора гиперпараметров для моделей. В этом процессе будет использоваться GridSearch.

Для логистической регрессии и CatBoost можно по сетке перебрать относительно большое количество гиперпараметров - регрессия самая по себе быстро обучается, а у CatBoost'а есть простая возможность производить обучение на GPU. Для LGBMClassifier тоже попробуем улучшить результат, но параметров будет меньше.

In [8]:
%%time
print('Логистическая регрессия.')
param_grid1 = [
    {'penalty': ['l1'], 'solver': ['newton-cg'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
    {'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear', 'sag', 'saga'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
]
grid_search(model1, param_grid1)

Логистическая регрессия.
Оптимальные параметры модели:


{'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}

Качество модели после обучения: 0.7655294554223555
F1-score на тестовой выборке: 0.7830578512396694
Wall time: 7min 39s


In [6]:
%%time
print('LGBMClassifier')
param_grid2 = {
    'n_estimators': [100, 200, 500],
    'learning_rate' : [0.1, 0.5, 1],
}
grid_search(model2, param_grid2)

LGBMClassifier
Оптимальные параметры модели:


{'learning_rate': 0.1, 'n_estimators': 500}

Качество модели после обучения: 0.7727236294046722
F1-score на тестовой выборке: 0.794950528829751
Wall time: 26min 30s


По неизвестной причине CatBoost при попытке произвести поиск по сетке через собственный метод grid_search падает с ошибкой (ошибка возникает только при использовании GPU, на CPU всё в порядке - но поиск происходит долго). Не самым элегантным, но более эффективным образом осуществим ручной перебор параметров в цикле.

In [7]:
%%time
print('CatBoostClassifier')

# Задаём начальные переменные f1_score
f1_score_current = 0
f1_score_best = 0.7

# Задаём сетки для разных гиперпараметров
learning_rate_list = [0.1, 0.5, 1]
depth_list = [3, 6, 9, 12]
l2_leaf_reg_list = [1, 3, 5, 7, 9]

for l2 in l2_leaf_reg_list:
    for lr in learning_rate_list:
        for d in depth_list:
            model3 = CatBoostClassifier(task_type='GPU', devices='0:1', verbose=False, iterations = 500,
                                        learning_rate = lr, depth = d, l2_leaf_reg = l2).fit(train_pool)
            f1_score_current = f1_score(y_true, model3.predict(test_pool))                            
            if f1_score_current > f1_score_best:
                f1_score_best = f1_score_current
                print('')
                print('При параметрах:', model3.get_params(), 'метрика F1 улучшилась')
                print('F1-score на тестовой выборке:', f1_score_current)

CatBoostClassifier

При параметрах: {'iterations': 500, 'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 1, 'verbose': False, 'task_type': 'GPU', 'devices': '0:1'} метрика F1 улучшилась
F1-score на тестовой выборке: 0.7047685834502103

При параметрах: {'iterations': 500, 'learning_rate': 0.1, 'depth': 9, 'l2_leaf_reg': 1, 'verbose': False, 'task_type': 'GPU', 'devices': '0:1'} метрика F1 улучшилась
F1-score на тестовой выборке: 0.7104895104895104

При параметрах: {'iterations': 500, 'learning_rate': 0.1, 'depth': 12, 'l2_leaf_reg': 5, 'verbose': False, 'task_type': 'GPU', 'devices': '0:1'} метрика F1 улучшилась
F1-score на тестовой выборке: 0.7157378198387663
Wall time: 32min 37s


## Выводы

Выбранные модели логистической регрессии, LGBM, CatBoost и случайного леса, уже "из коробки" показали неплохие результаты не смотря на дисбаланс классов. Единственная откровенно **неудачная модель - случайный лес**, который оказался крайне медлительным и неэффективны для задачи классификации текстов.

А вот для остальных моделей, подбор гиперпараметров помог закрепить результат и повысить качество ещё на несколько пунктов. Правда **CatBoost так и не смог преодолеть необохдимый порог F1-меры** не меньше 0.75. Возможно сказалось сниженное количество итераций, возможно неэффективен перебор в цикле, возможно просто не угадали с набором параметров - сказать наверняка сложно, но к счастью есть другие модели.

**Лучшее качество показал LGBMClassifier** - показатель F1-меры достиг 0.77 при обучении и 0.79 на тестовой выборке. Скорость модели получилась выше, чем у CatBoost'а (правда и параметров было меньше), но стоит помнить - работала модель на CPU, а ведь потенциально её можно запустить и на GPU.

**Логистическая регрессия** не сильно отстала в качестве от градиентных бустингов, и также преодолела необходимый порог, при этом оказалась в 3 раза быстрее, так что если в приоритете оперативность - **вариант тоже отличный.**